# CUDA programming in Julia

## CUDA driver API - CUDAdrv

CUDAdrv and CUDAnative are two CUDA-related packages. CUDAdrv is built upon the low-level CUDA driver API. Additionally, CUDAdrv provides automatic memory management. PTX modules are allowed to be compiled and loaded by CUDAdrv.

In [1]:
using CUDAdrv

### Device information

In [2]:
CUDAdrv.vendor()

"NVIDIA"

In [3]:
dev = CuDevice(0)
CUDAdrv.name(dev)

"TITAN Xp"

### CUDA version

In [4]:
CUDAdrv.version()

v"10.0.0"

### CUDA computation information

[Compute Capabilities](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities)

In [5]:
CUDAdrv.capability(dev)

v"6.1.0"

In [6]:
CUDAdrv.warpsize(dev)

32

### CuContext

In [7]:
dev = CuDevice(0)
ctx = CuContext(dev)

CuContext(Ptr{Nothing} @0x00000000037cd0b0, true, true)

In [8]:
A = zeros(Float32, 3, 4)
d_A = Mem.upload(A);

In [9]:
typeof(d_A)

CUDAdrv.Mem.Buffer

In [10]:
CUDAdrv.Mem.total()

12788498432

In [11]:
CUDAdrv.Mem.info()

(12259819520, 12788498432)

In [12]:
CUDAdrv.Mem.used()

528678912

In [13]:
CUDAdrv.Mem.free()

12259819520

In [14]:
destroy!(ctx)

## Low-level CUDA programming - CUDAnative

CUDAnative supports the native CUDA programming, and it compiles and executes native Julia kernels directly to CUDA device. CUDAnative is the alternative of CUDART (CUDA RunTime library) and NVRTC (NVIDIA RunTime Compilation library for CUDA C++) in Julia.

In [15]:
using CUDAnative, CuArrays

### Write kernels

> Kernels are functions or subroutines run on CUDA.

In [16]:
function vadd(a, b, c)
    i = (blockIdx().x-1) * blockDim().x + threadIdx().x
    c[i] = a[i] + b[i]
    return nothing
end

vadd (generic function with 1 method)

In [17]:
a = rand(Float32, 10)
b = rand(Float32, 10)
ad = CuArray(a)
bd = CuArray(b)

10-element CuArray{Float32,1}:
 0.60174704
 0.39030552
 0.4456973 
 0.41883242
 0.89954865
 0.3787924 
 0.645728  
 0.57290137
 0.8615589 
 0.28575182

In [18]:
c = zeros(Float32, 10)
cd = CuArray(c)

10-element CuArray{Float32,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [19]:
@cuda threads=10 vadd(ad, bd, cd)

In [20]:
cd

10-element CuArray{Float32,1}:
 1.0741465 
 0.41989088
 0.75328994
 1.3015609 
 1.5043014 
 0.7520429 
 1.0103626 
 0.6099775 
 1.0527846 
 0.52501607

In [21]:
Array(cd)

10-element Array{Float32,1}:
 1.0741465 
 0.41989088
 0.75328994
 1.3015609 
 1.5043014 
 0.7520429 
 1.0103626 
 0.6099775 
 1.0527846 
 0.52501607

#### Note

Julia convention is that matrices are stored in column-major order, whereas C (and CUDA) use row-major. The linear sequence of addresses is the same between main memory and the GPU.

## High-level programming - CuArrays

In [22]:
using CuArrays

### Construct CuArray

In [23]:
a = cu([1, 2, 3])

3-element CuArray{Float32,1}:
 1.0
 2.0
 3.0

In [24]:
a = fill(CuArray{Int}, 0, (100,))

100-element CuArray{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

### Self-defined type in CuArray

Make self-defined type over CuArray.

In [25]:
struct Point
    x::Float32
    y::Float32
end

In [26]:
Base.convert(::Type{Point}, x::NTuple{2, Any}) = Point(x[1], x[2])

In [27]:
b = cu(Point[(1, 2), (4, 3), (2, 2)])

3-element CuArray{Point,1}:
 Point(1.0f0, 2.0f0)
 Point(4.0f0, 3.0f0)
 Point(2.0f0, 2.0f0)

### CuArray calculation

Support for broadcasting.

In [28]:
A = cu([1, 2, 3])
B = cu([1, 2, 3])
C = cu(rand(Float32, 3))

3-element CuArray{Float32,1}:
 0.9871224 
 0.5199164 
 0.12411082

In [29]:
result = A .+ B .- C

3-element CuArray{Float32,1}:
 1.0128776
 3.4800835
 5.8758893

### Function applied on CuArray

In [30]:
double(a::T) where T = a * convert(T, 2)
result .= double.(A)

3-element CuArray{Float32,1}:
 2.0
 4.0
 6.0

### Hybrid computation

Hybrid calculation with operands from CPU Array and CUDA CuArray.

In [31]:
Base.:(+)(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)

In [32]:
cu_points = cu(Point[(1, 2), (4, 3), (2, 2)])
result = cu_points .+ Ref(Point(2, 2))

3-element CuArray{Point,1}:
 Point(3.0f0, 4.0f0)
 Point(6.0f0, 5.0f0)
 Point(4.0f0, 4.0f0)

#### More operations supported by GPUArrays

* Conversions and `copy!` to CPU arrays
* multi-dimensional indexing and slicing `xs[1:2, 5, :]`
* permutedims
* Concatenation (`vcat(x, y)`, `cat(3, xs, ys, zs)`)
* map, fused broadcast (`zs .= xs.^2 .+ ys .* 2`)
* `fill(CuArray, 0f0, dims)`, `fill!(gpu_array, 0)`
* Reduction over dimensions (`reduce(+, xs, dims = 3)`, `sum(x -> x^2, xs, dims = 1)`)
* Reduction to scalar (`reduce(*, xs)`, `sum(xs)`, `prod(xs)`)
* Various BLAS operations (matrix*matrix, matrix*vector)
* FFTs, using the same API as julia's FFT

## Reference

[An Introduction to GPU Programming in Julia](https://nextjournal.com/sdanisch/julia-gpu-programming)